In [ ]:
#!/usr/bin/env python3

#!pip insall sitemap_tree_for_homepage
#!pip insall pandas

import csv, requests
from usp.tree import sitemap_tree_for_homepage
import pandas as pd

In [ ]:
tree = sitemap_tree_for_homepage('https://we.dnomia.com')

pageDetails = [[
    page.url,
    page.last_modified.isoformat('#','hours').split('#')[0] if page.last_modified else None,
    float(page.priority) if page.priority else None] for page in tree.all_pages()]

with open('pages.csv', 'w+', newline='') as fl:
    write = csv.writer(fl)
    write.writerow(['URL', 'LastModified', 'Priority'])
    write.writerows(pageDetails)

print(f'{len(pageDetails)} rows founded!')

In [ ]:
with open('pages.csv') as csvFile:
    csvReader = csv.reader(csvFile, delimiter=',')
    print([row[0] for row in csvReader)][1:10)

In [ ]:
df = pd.read_csv('pages.csv')
df.head()

In [ ]:
# https://pythontic.com/pandas/dataframe-attributes/introduction
print(df.shape)
print(df.columns)
print(df.describe)

In [ ]:
for url in df['URL']:
    response = requests.get(url)
    df['Status'] = response.status_code
    df['Redirected'] = response.is_redirect
    df['PermanentRedirection'] = response.is_permanent_redirect
    df['IssueReason'] = response.reason if response.reason != 'OK' else None

In [ ]:
df.head()

In [ ]:
print(df.columns)
print(list(set(df['Priority'])))
print(list(set(df['Status'])))
print(list(set(df['Redirected'])))
print(list(set(df['PermanentRedirection'])))
print(list(set(df['IssueReason'])))

In [ ]:
df.loc[df.Priority > 0.6]

In [ ]:
# df['Status'].value_counts()
df[['URL', 'Status']].head(10)

In [ ]:
df['Status'] = (pd
    .Categorical(
        df['Status'],
        categories=list(set(df['Status'])),
        ordered=False))

df['Priority'] = (pd
    .Categorical(
        df['Priority'],
        categories=[float(i/10) for i in range(0,10,1)],
        ordered=True))

df['Redirected'] = (pd
    .Categorical(
        df['Redirected'],
        categories=list(set(df['Redirected'])),
        ordered=False))

df['PermanentRedirection'] = (pd
    .Categorical(
        df['PermanentRedirection'],
        categories=list(set(df['PermanentRedirection'])),
        ordered=False))

In [ ]:
df[['URL', 'Status','Priority']].sample(5)

In [ ]:
df.to_csv('sitemap.zip', index=False, compression=dict(method='zip',archive_name='pages-new.csv'))